# Table of Content

- [8.1 Changing the String Representation of Instances](#8.1)
- [8.5 Encapsulating Names in a Class](#8.5)
- [8.6 Creating Managed Attributes](#8.6)
- [8.7 Calling a Method a Parent Class](#8.7)
- [8.9 Creating a New Kind of Class or Instance Attribute](#8.9)
- [8.10 Using Lazily Computed Properties](#8.10)
- [8.16 Defining More Than One Constructor in a Class](#8.16)
- [8.18 Extending Classes with Mixinxs](#8.18)
- [8.20 Calling a method on an Object Given the Name As a String](#8.20)

---
## <a name="8.1"></a> 8.1 Changing the String Representation of Instances

### Solution
Define `__str__` or `__repr__`

### Discussion
Using `format()` with `self` to implement `__repr__`  
(`!r` indicates that the output of `__repr__` should be used instead of `__str__`)

In [1]:
class Class:
    def __init__(self):
        self.x = 1
        self.y = 2
    
    def __repr__(self):
        return 'x: {0.x!r} y: {0.y}'.format(self)
    

c = Class()
print(c)

x: 1 y: 2


---
## <a name="8.5"></a> 8.5 Encapsulating Names in a Class

### Solution

- `_name`: Internal implementation
- `__name`: Private. These attributes will be renamed as `_class__name`
- `name_`: Used when `name` clashes with reserved word

### Discussion
Most time, `_name` should be enough.  
`__name` should be used when your code will involve subcalssing and these attributes should be hidden from subclasses

---
## <a name="8.6"></a> 8.6 Creating Managed Attributes

### Solution

In [2]:
class Person:
    def __init__(self, first_name):
        self.first_name = first_name
        
    @property
    def first_name(self):
        return self._first_name
    
    @first_name.setter
    def first_name(self, value):
        if not isinstance(value, str):
            raise TypeError('Expected a string')
        self._first_name = value
        
    @first_name.deleter
    def first_name(self):
        raise ArithmeticError("Can't delete attribute")

        
p = Person('Wei')
p.first_name

'Wei'

### Discussion

- ***Properties should only be used in cases where you actually need to perform extra processing on attribute access.*** Sometimes programmers coming from languages might feel that all access should be handled by getters and seeters

There are 3 disadvantage of doing this
1. Make your code verbose and confusing
2. Make your code slower
3. Gain no real design benefit


- ***DO NOT write Python code that features a lot of repetitive property definitions.***  
Readd 8.9 and 9.21

## <a name="8.7"></a> 8.7 Calling a Method a Parent Class

### Discussion
Be careful when using `super()`

In [3]:
class A:
    def spam(self):
        print('A.spam')
        super().spam()

a = A()
a.spam()

A.spam


AttributeError: 'super' object has no attribute 'spam'

In [4]:
class B:
    def spam(self):
        print('B.spam')
        
class C(A, B):
    pass

c = C()
c.spam()

A.spam
B.spam


`super().spam()` in A envokes the spam in B which might not be expected.    
It's due to the MRO of C

In [5]:
C.__mro__

(__main__.C, __main__.A, __main__.B, object)

There are some general rules should be followed when using `super()`
1. All method with the same name in an inheritance hierarchy should have a compatible calling signature (i.e. same number of arguments, argument names)
2. Ensures that `super()` won't get tripped up if it tires to invloke a method on a class that's not direct parent
3. The topmost class should provide an implementation of the method so that the chain of lookups that occur along the MRO get terminated by an actual method

Here is a worth reading artible related to super
[Python’s super() considered super!](https://rhettinger.wordpress.com/2011/05/26/super-considered-super/)

---
## <a name="8.9"></a> 8.9 Creating a New Kind of Class or Instance Attribute

### Solution
Descriptor: A class that implements the three core attribute access operations (get, set, delete)

In [6]:
class Integer:
    def __init__(self, name):
        self.name = name
        
    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            return instance.__dict__[self.name]
        
    def __set__(self, instance, value):
        if not isinstance(value, int):
            raise TypeError('Expected an int')
        else:
            instance.__dict__[self.name] = value
        
    def __delete__(self, instance):
        del instance.__dict__[self.name]

In [7]:
class Point:
    x = Integer('x')
    y = Integer('y')
    def __init__(self, x, y):
        self.x = x
        self.y = y

p = Point(2, 3)
p = Point('2', '3')

TypeError: Expected an int

### Discussion

**Note that descriptor can only be defined at the class level, not on per-instance basis**

If you simply want to customize the access of a single attribute of a specific class, the use of property might be suitable.  
Read [8.6](#8.6)

---
## <a name="8.10"></a> 8.10 Using Lazily Computed Properties
You'd like to define a read-only attribute as a property that only gets computed on access.  
However, once accessed, you'd like the value to be cached and not recomputed on each access

### Solution

In [8]:
import math


class lazyproperty:
    def __init__(self, func):
        self.func = func
    
    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            value = self.func(instance)
            setattr(instance, self.func.__name__, value)
            return value


class Circle:
    def __init__(self, radius):
        self.radius = radius
        
    @lazyproperty
    def area(self):
        print('Computing area')
        return math.pi * self.radius * 2

    
c = Circle(4.0)
print(c.area)
print(c.area)

Computing area
25.132741228718345
25.132741228718345


Note that the message "Computing area" only be printed once

One downside to this is that the computed value becomes mutable after it's created  

---
## <a name="8.16"></a> 8.16 Defining More Than One Constructor in a Class

### Solution

In [9]:
import time


class Date:
    # Primary Constructor
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day
    
    # Alternate constructor
    @classmethod
    def today(cls):
        t = time.localtime()
        return cls(t.tm_year, t.tm_mon, t.tm_mday)
    
d1 = Date(2012, 12, 21)
d2 = Date.today()

### Discussion

When defining a class with multiple constructors, `__init__()` should be as simple as possible

---
## <a name="8.18"></a> 8.18 Extending Classes with Mixinxs

### Solution

In [10]:
class LoggedMappingMixin:
    __slots__ = ()
    
    def __getitem__(self, key):
        print('Getting ', str(key))
        return super().__getitem__(key)
    
    def __setitem__(self, key, value):
        print('Setting {} = {!r}'.format(key, value))
        return super().__setitem__(key, value)
    
    def __delitem__(self, key):
        print('Deleting ', str(key))
        return super().__delitem__(key)
    
class LoggedDict(LoggedMappingMixin, dict):
    pass

d = LoggedDict()
d['a'] = 1

Setting a = 1


### Discussion

#### Principle for Mixin Classes
- Never meant to be instantiated directly 
- Typically have no state of their own
    - no `__init__()`
    - no instance variables
    - **`__slots__()`** is meant to serve as a strong hint that the mixin classes do not have their own instance data
- Use of **`super()`** is essential

### Alternative Solution
decorator

In [11]:
def LoggedMapping(cls):
    cls_getitem = cls.__getitem__
    cls_setitem = cls.__setitem__
    cls_delitem = cls.__delitem__
    
    def __getitem__(self, key):
        print('Getting ', str(key))
        return cls_getitem(self, key)
    
    def __setitem__(self, key, value):
        print('Setting {} = {!r}'.format(key, value))
        return cls_setitem(self, key, value)
    
    def __delitem__(self, key):
        print('Deleting ', str(key))
        return cls_delitem(self, key)
    
    cls.__getitem__ = __getitem__
    cls.__setitem__ = __setitem__
    cls.__delitem__ = __delitem__
    return cls

@LoggedMapping
class LoggedDict(dict):
    pass

d = LoggedDict()
d['a'] = 1

Setting a = 1


---
## <a name="8.20"></a> 8.20 Calling a method on an Object Given the Name As a String
### Solution

In [12]:
import math


class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def distance(self, x, y):
        return math.hypot(self.x -x , self.y - y)
    

p = Point(2, 3)

- getattr()

In [13]:
getattr(p, 'distance')(0,0)

3.605551275463989

- operator.methodcaller()

In [14]:
import operator

operator.methodcaller('distance', 0, 0)(p)

3.605551275463989

### Discussion
This can be useful when emulating case statements or variants of the visitor pattern